## MODNet


In [ ]:
import os

# clone the repository
if not os.path.exists('MODNet'):
  !git clone https://github.com/ZHKKKe/MODNet

%cd MODNet/

# dowload the pre-trained ckpt for image matting
pretrained_ckpt = 'pretrained/modnet_photographic_portrait_matting.ckpt'
if not os.path.exists(pretrained_ckpt):
  !gdown --id 1mcr7ALciuAsHCpLnrtG_eop5-EYhbCmz \
          -O pretrained/modnet_photographic_portrait_matting.ckpt


## Inference

<p align="justify">Run the following command for alpha matte prediction:</p>

In [ ]:
!python -m demo.image_matting.colab.inference \
        --input-path data/original \
        --output-path data/masks \
        --ckpt-path ./pretrained/modnet_photographic_portrait_matting.ckpt


In [27]:
import numpy as np
from PIL import Image
import cv2  

# extract backgrounds from input using mask
def combined_display(image, matte, image_name):
    w, h, c = image.shape
    rw, rh = 800, int(h * 800 / (3 * w))
    foreground = cv2.bitwise_and(image, image, mask=matte)
    (thresh, matte) = cv2.threshold(matte, w, h, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    thresh = 255 - thresh
    matte = cv2.threshold(matte, thresh, 255, cv2.THRESH_BINARY)[1]
    foreground = cv2.bitwise_and(image, image, mask=matte)

    cv2.imwrite('../data/person_dataset/'+image_name, foreground)

input_folder = '../data/original/'
output_folder = '../data/masks/'
# visualize all images
image_names = os.listdir(input_folder)

for image_name in image_names:
    matte_name = image_name.split('.')[0] + '.png'
    image = cv2.imread(os.path.join(input_folder, image_name))
    matte = cv2.imread(os.path.join(output_folder, matte_name),cv2.IMREAD_GRAYSCALE)
    #  matte = cv2.cvtColor(matte, cv2.COLOR_GRAY2BGR)

    
    combined_display(image, matte, matte_name)


## Download Results

<p align="justify">Download the Zip package of predicted alpha mattes:</p>

In [ ]:
zip_filename = 'matte.zip'
import os 

if os.path.exists(zip_filename):
  os.remove(zip_filename)

os.system(f"zip -r -j {zip_filename} {output_folder}/*")
files.download(zip_filename)